# SimpleQA for the Bulgarian models

<h4>Imports</h4>

In [11]:
import pandas as pd
from openai import OpenAI
import dotenv
import requests
import json

<h3>Prompt for the Simple QA</h3>

In [ ]:
# This grader template is taken directly form "Measuring short-form factuality in large language models"
GRADER_TEMPLATE_BG = """
Твоята задача е да погледнеш даден въпрос, неговият правилен отговор и предвиден отговор. След това трябва да определиш дали предведения отговор е ["ВЕРЕН", "НЕВЕРЕН", "НЕОПИТАН"] на база на правилния отговор.
Като за начало, ще ти дам пример за всеки един отговор ["ВЕРЕН", "НЕВЕРЕН", "НЕОПИТАН"], след което твоята задача ще е да определиш нов пример.

Следните са примери за ВЕРНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия Обама и Саша Обама.
Предвиден отговор 1: саша и малия обама
Предвиден отговор 2: повечето хора биха казали Малия и Саша, но не съм сигурен и ще трябва да проверя.
Предвиден отговор 3: Барак Обама има две дъщери. Имената им са Малия Ан и Наташа Мариан, но те се наричат обикновено Малия Обама и Саша Обама. Малия е родена на 4 юли 1998 г., а Саша е родена на 10 юни 2001 г.
Предвиден отговор 4: Malia Obama and Sasha Obama.
Предвиден отговор 5: Sasha and Malia.
‘‘‘
Тези предвидени отговори са ВЕРНИ, защото:
- Съдържат пълната информация от правилния отговор.
- Не съдържат информация, която противоречи на правилния отговор.
- Само семантичното значение е важно; главни букви, пунктуация, граматика и редът не са важни.
- Налучкването е позволено, при условие че правилният отговор е напълно включен и отговорът не съдържа неправилна информация или противоречия.
- Имената може да бъдат написани както на кирилица (напр. Малия Обама), така и на латиница (напр. Malia Obama). Това не променя верността на отговора.
Следните са примери за НЕВЕРНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия и Саша
Предвиден отговор 1: Малия.
Предвиден отговор 2: Малия, Саша и Сюзън.
Предвиден отговор 3: Барак Обама няма деца.
Предвиден отговор 4: Мисля, че е Малия и Саша. Или може би е Малия и Джаки. Или може би е Джоуи и Малия.
Предвиден отговор 4: Въпреки че не знам техните точни имена, мога да ви кажа, че Барак Обама има три деца.
Предвиден отговор 5: Възможно е да имате предвид Бетси и Оливия. Въпреки това, трябва да уточните допълнителни подробности с актуализирана информация, ако е необходимо. Това ли е правилният отговор?
Предвиден отговор 6: Може би детето на Обама се казва Джеймс. Въпреки това е препоръчително да потвърдите най-точната и актуална информация, тъй като това може да се промени с времето. Този модел не винаги отразява най-актуалната информация.
‘‘‘
Тези предвидени отговори са НЕВЕРНИ, защото:
-Съдържат неправилна информация или самият отговор противоречи на правилния отговор. Неправилни отговори, които имат някакво налучкване (например "възможно е", "макар и не съм сигурен, мисля") също се считат за невярни.
Следните са примери за НЕОПИТАНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия и Саша
Предвиден отговор 1: Не знам.
Предвиден отговор 2: Трябва му повече контекст за кой Обама говорите.
Предвиден отговор 3: Без да търся в интернет, не мога да отговоря на този въпрос. Въпреки това, мога да ви кажа, че Барак Обама има две деца.
Предвиден отговор 4: Барак Обама има две деца. Знам, че едното се казва Малия, но не съм сигурен за другото.
‘‘‘
Тези предвидени отговори са НЕОПИТАНИ, защото:
-Важната информация от правилния отговор не е включена в отговора.
-Няма изявления в отговора, които противоречат на правилния отговор.
Също така, обърни внимание на следните неща:
-За определянето на въпроси където правилният отговор е число, предвиденият отговор трябва да бъде верен до последната значима цифра в правилния отговор.
Например, ако въпросът е "Колко цитати има статията за Трансформърс?" с правилен отговор "120 хиляди".
-Предвидените отговори "120 хиляди", "124 хиляди" и "115 хиляди" са ВЕРНИ.
-Предвидените отговори "100 хиляди" и "113 хиляди" са НЕВЕРНИ.
-Предвидените отговори "около 100 хиляди" и "повече от 50 хиляди" се считат за НЕОПИТАНИ, защото нито потвърждават, нито противоречат на правилния отговор.
-Правилният отговор може да съдържа повече информация от въпроса. В такива случаи, предвиденият отговор трябва да съдържа само информацията, която е въпроса.
-Например, ако въпросът е "В коя серия Дерек и Мередит се омъжиха в Анатомията на Грей?" с правилен отговор "Сезон 7, Епизод 20: Бяла сватба".
И "Сезон 7, Епизод 20" и "Бяла сватба" ще се считат за ВЕРНИ.
.
-Не санкционирай предвидените отговори, ако пропускат информация, която може да бъде лесно извлечена от въпроса.
-Например, ако въпросът е "В кой град се намира OpenAI?" и правилният отговор е "Сан Франциско, Калифорния". Предвиденият отговор "Сан Франциско" ще се счита за ВЕРЕН, въпреки че не съдържа "Калифорния".
-Друг пример е въпросът "Каква награда спечели Ръководство за предварително обучение на данни:: Измерване на датирането на база данни, покритието на домейна, качеството и токсичността на NAACL’24?", като правилният отговор е "Награда за най-добра статия". Предвиденият отговор "Най-добра статия" ще се счита за ВЕРЕН, защото "награда" се предполага във въпроса.
-За въпроса "Каква е височината на Джейсън Уей в метри?", правилният отговор е "1.73 м". Предвиденият отговор "1.75" ще се счита за ВЕРЕН, защото въпросът специфицира метри.
-За въпроса " Какво е името на съпругата на Барак Обама?", правилният отговор е "Мишел Обама". Предвиденият отговор "Мишел" ще се счита за ВЕРЕН, защото фамилното име се предполага.
-Не санкционирай за грешки в изписването на имената на хора, ако е ясно, че се отнася до същото име.
-Например, ако правилният отговор е "Хюнг Уон Чунг", можеш да считаш следните предвидени отговори за верни: "Хююнг Уон Чунг", "Хюнгуон Чунг" или "Хюн Уон Чунг".
Ето нов пример. Просто отговори с ВЕРЕН, НЕВЕРЕН, НЕОПИТАН. Не се извинявай или коригирай, ако има грешка; просто се опитваме да определим отговора.
‘‘‘
Въпрос: {question}
Правилен отговор: {target}
Предвиден отговор: {predicted_answer}
‘‘‘
Определи предвидения отговор на този нов въпрос като един от:
A: ВЕРЕН
B: НЕВЕРЕН
C: НЕОПИТАН
Просто върни буквите "A", "B" или "C", без текст около тях.
""".strip()



GRADER_TEMPLATE_EN = """
Your job is to look at a question, a gold target, and a predicted answer, and then assign a
grade of either ["CORRECT", "INCORRECT", "NOT_ATTEMPTED"].
First, I will give examples of each grade, and then you will grade a new example.
The following are examples of CORRECT predicted answers.
‘‘‘
Question: What are the names of Barack Obama’s children?
Gold target: Malia Obama and Sasha Obama
Predicted answer 1: sasha and malia obama
Predicted answer 2: most people would say Malia and Sasha, but I’m not sure and would have to
double check
Predicted answer 3: Barack Obama has two daughters. Their names are Malia Ann and Natasha
Marian, but they are commonly referred to as Malia Obama and Sasha Obama. Malia was born
on July 4, 1998, and Sasha was born on June 10, 2001.
‘‘‘
These predicted answers are all CORRECT because:
- They fully contain the important information in the gold target.
- They do not contain any information that contradicts the gold target.
- Only semantic meaning matters; capitalization, punctuation, grammar, and order don’t
matter.
- Hedging and guessing are permissible, provided that the gold target is fully included
and the response contains no incorrect information or contradictions.
The following are examples of INCORRECT predicted answers.
‘‘‘
Question: What are the names of Barack Obama’s children?
Gold target: Malia and Sasha
Predicted answer 1: Malia.
Predicted answer 2: Malia, Sasha, and Susan.
Predicted answer 3: Barack Obama does not have any children.
Predicted answer 4: I think it’s either Malia and Sasha. Or it could be Malia and Jackie. Or
it could be Joey and Malia.
Predicted answer 4: While I don’t know their exact names, I can tell you that Barack Obama
has three children.
Predicted answer 5: It’s possible you may mean Betsy and Olivia. However, you should clarify
further details with updated references if necessary. Is that the correct answer?
Predicted answer 6: It may be the case that Obama’s child is named James. However, it’s
recommended to confirm the most accurate and updated information since this could change
over time. This model may not always reflect the most current information.
‘‘‘
These predicted answers are all INCORRECT because:
- A factual statement in the answer contradicts the gold target. Incorrect statements
that have some hedging (e.g., "it is possible that", "although i’m not sure, i think
") are also considered incorrect.
The following are examples of NOT_ATTEMPTED predicted answers.
‘‘‘
Question: What are the names of Barack Obama’s children?
Gold target: Malia and Sasha
Predicted answer 1: I don’t know.
Predicted answer 2: I need more context about which Obama you are talking about.
Predicted answer 3: Without researching the web, I cannot answer this question. However, I
can tell you that Barack Obama has two children.
11
Predicted answer 4: Barack Obama has two children. I know that one of them is Malia, but I’m
not sure about the other one.
‘‘‘
These predicted answers are all NOT_ATTEMPTED because:
- The important information in the gold target is not included in the answer.
- No statements in the answer contradict the gold target.
Also note the following things:
- For grading questions where the gold target is a number, the predicted answer needs to be
correct to the last significant figure in the gold answer. For example, consider a
question "How many citations does the Transformer Paper have?" with gold target "120k".
- Predicted answers "120k", "124k", and 115k" are all CORRECT.
- Predicted answers "100k" and "113k" are INCORRECT.
- Predicted answers "around 100k" and "more than 50k" are considered NOT_ATTEMPTED
because they neither confirm nor contradict the gold target.
- The gold target may contain more information than the question. In such cases, the
predicted answer only needs to contain the information that is in the question.
- For example, consider the question "What episode did Derek and Meredith get legally
married in Grey’s Anatomy?" with gold target "Season 7, Episode 20: White Wedding".
Either "Season 7, Episode 20" or "White Wedding" would be considered a CORRECT answer
.
- Do not punish predicted answers if they omit information that would be clearly inferred
from the question.
- For example, consider the question "What city is OpenAI headquartered in?" and the gold
target "San Francisco, California". The predicted answer "San Francisco" would be
considered CORRECT, even though it does not include "California".
- Consider the question "What award did A pretrainer’s guide to training data: Measuring
the effects of data age, domain coverage, quality, & toxicity win at NAACL ’24?", the
gold target is "Outstanding Paper Award". The predicted answer "Outstanding Paper"
would be considered CORRECT, because "award" is presumed in the question.
- For the question "What is the height of Jason Wei in meters?", the gold target is "1.73
m". The predicted answer "1.75" would be considered CORRECT, because meters is
specified in the question.
- For the question "What is the name of Barack Obama’s wife?", the gold target is "
Michelle Obama". The predicted answer "Michelle" would be considered CORRECT, because
the last name can be presumed.
- Do not punish for typos in people’s name if it’s clearly the same name.
- For example, if the gold target is "Hyung Won Chung", you can consider the following
predicted answers as correct: "Hyoong Won Choong", "Hyungwon Chung", or "Hyun Won
Chung".
Here is a new example. Simply reply with either CORRECT, INCORRECT, NOT ATTEMPTED. Don’t
apologize or correct yourself if there was a mistake; we are just trying to grade the
answer.
‘‘‘
Question: {question}
Gold target: {target}
Predicted answer: {predicted_answer}
‘‘‘
Grade the predicted answer of this new question as one of:
A: CORRECT
B: INCORRECT
C: NOT_ATTEMPTED
Just return the letters "A", "B", or "C", with no text around it.
""".strip()


GRADER_TEMPLATE_PL = """
Twoim zadaniem jest przeanalizować pytanie, wzorcową (docelową) odpowiedź oraz przewidywaną odpowiedź, a następnie przydzielić ocenę spośród: ["CORRECT" (POPRAWNA), "INCORRECT" (NIEPOPRAWNA), "NOT_ATTEMPTED" (NIEPODJĘTA)].
Najpierw podam przykłady każdej z ocen, a następnie będziesz musiał ocenić nowy przykład.

Oto przykłady odpowiedzi ocenianych jako CORRECT (POPRAWNA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia Obama i Sasha Obama
Przewidywana odpowiedź 1: sasha i malia obama
Przewidywana odpowiedź 2: większość powiedziałaby Malia i Sasha, ale nie jestem pewien i musiałbym to sprawdzić
Przewidywana odpowiedź 3: Barack Obama ma dwie córki. Mają na imię Malia Ann i Natasha Marian, lecz powszechnie nazywa się je Malia Obama i Sasha Obama. Malia urodziła się 4 lipca 1998, a Sasha 10 czerwca 2001.
‘‘‘
Te przewidywane odpowiedzi są CORRECT, ponieważ:

W pełni zawierają ważne informacje z wzorcowej odpowiedzi.
Nie zawierają żadnych informacji sprzecznych z wzorcową odpowiedzią.
Liczy się jedynie znaczenie semantyczne; wielkość liter, interpunkcja, gramatyka i kolejność nie mają znaczenia.
Określenia niepewności i przypuszczenia są dopuszczalne, o ile wzorcowa odpowiedź jest w pełni zawarta i nie ma niepoprawnych informacji lub sprzeczności.
Oto przykłady odpowiedzi ocenianych jako INCORRECT (NIEPOPRAWNA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia i Sasha
Przewidywana odpowiedź 1: Malia.
Przewidywana odpowiedź 2: Malia, Sasha i Susan.
Przewidywana odpowiedź 3: Barack Obama nie ma żadnych dzieci.
Przewidywana odpowiedź 4: Myślę, że to albo Malia i Sasha. Albo może Malia i Jackie. Albo może Joey i Malia.
Przewidywana odpowiedź 4: Chociaż nie znam ich dokładnych imion, mogę powiedzieć, że Barack Obama ma trójkę dzieci.
Przewidywana odpowiedź 5: Być może chodzi o Betsy i Olivię. Jednak powinieneś sprawdzić bardziej aktualne źródła, aby to potwierdzić. Czy to poprawna odpowiedź?
Przewidywana odpowiedź 6: Możliwe, że dziecko Obamy to James. Jednak zaleca się potwierdzić najdokładniejsze i najbardziej aktualne informacje, ponieważ to może się zmieniać z czasem. Ten model może nie zawsze odzwierciedlać najbardziej aktualne dane.
‘‘‘
Te przewidywane odpowiedzi są INCORRECT, ponieważ:

Znajdują się w nich stwierdzenia sprzeczne z wzorcową odpowiedzią. Niepoprawne informacje, nawet podane z zastrzeżeniem (np. "możliwe, że", "choć nie jestem pewien, to myślę..."), uznawane są za niepoprawne.
Oto przykłady odpowiedzi ocenianych jako NOT_ATTEMPTED (NIEPODJĘTA).
‘‘‘
Pytanie: Jak mają na imię dzieci Baracka Obamy?
Wzorcowa odpowiedź: Malia i Sasha
Przewidywana odpowiedź 1: Nie wiem.
Przewidywana odpowiedź 2: Potrzebuję więcej kontekstu o którego Obamę chodzi.
Przewidywana odpowiedź 3: Bez przeszukiwania sieci nie mogę odpowiedzieć. Mogę jednak powiedzieć, że Barack Obama ma dwoje dzieci.
Przewidywana odpowiedź 4: Barack Obama ma dwoje dzieci. Wiem, że jedna z nich to Malia, ale nie jestem pewien co do drugiej.
‘‘‘
Te przewidywane odpowiedzi są NOT_ATTEMPTED, ponieważ:

Ważne informacje z wzorcowej odpowiedzi nie są zawarte w odpowiedzi.
W odpowiedzi nie ma stwierdzeń sprzecznych z wzorcową odpowiedzią.
Zwróć uwagę na poniższe kwestie:

Przy ocenianiu pytań, gdzie wzorcowa odpowiedź jest liczbą, przewidywana odpowiedź musi być poprawna co do ostatniej znaczącej cyfry wzorcowej odpowiedzi.
Na przykład, przy pytaniu "Ile cytowań ma praca Transformer Paper?" z odpowiedzią wzorcową "120k":

Odpowiedzi "120k", "124k" i "115k" są CORRECT.
Odpowiedzi "100k" i "113k" są INCORRECT.
Odpowiedzi "około 100k" i "ponad 50k" są NOT_ATTEMPTED, ponieważ ani nie potwierdzają, ani nie zaprzeczają wzorcowej odpowiedzi.
Wzorcowa odpowiedź może zawierać więcej informacji niż pytanie. W takich przypadkach przewidywana odpowiedź musi jedynie zawierać te informacje, które wymaga pytanie.
Na przykład, przy pytaniu "W którym odcinku Derek i Meredith wzięli legalny ślub w Grey’s Anatomy?" ze wzorcową odpowiedzią "Sezon 7, Odcinek 20: White Wedding":

Zarówno "Sezon 7, Odcinek 20" jak i "White Wedding" są uznawane za CORRECT.
Nie karz za pominięcie informacji, które można wywnioskować z pytania.
Na przykład, przy pytaniu "W jakim mieście mieści się siedziba OpenAI?" ze wzorcową odpowiedzią "San Francisco, Kalifornia":

Odpowiedź "San Francisco" jest uznawana za CORRECT, nawet jeśli nie zawiera "Kalifornia".
Rozważ pytanie "Jaką nagrodę zdobył artykuł ‘A pretrainer’s guide to training data: Measuring the effects of data age, domain coverage, quality, & toxicity’ na NAACL ’24?" ze wzorcową odpowiedzią "Outstanding Paper Award".

Odpowiedź "Outstanding Paper" jest uznawana za CORRECT, ponieważ słowo "award" można wywnioskować z pytania.
Przy pytaniu "Jaki jest wzrost Jasona Wei w metrach?" ze wzorcową odpowiedzią "1.73 m":

Odpowiedź "1.75" jest uznawana za CORRECT, ponieważ w pytaniu podano jednostkę (metry).
Przy pytaniu "Jak nazywa się żona Baracka Obamy?" ze wzorcową odpowiedzią "Michelle Obama":

Odpowiedź "Michelle" jest CORRECT, ponieważ można założyć, że chodzi o jej nazwisko Obama.
Nie karz za literówki w imionach i nazwiskach, jeśli wyraźnie chodzi o tę samą osobę.
Na przykład, jeśli wzorcowa odpowiedź to "Hyung Won Chung", to poniższe odpowiedzi są poprawne: "Hyoong Won Choong", "Hyungwon Chung" czy "Hyun Won Chung".

Oto nowy przykład. Wystarczy, że odpowiesz "A", "B" lub "C". Nie przepraszaj i nie poprawiaj się, jeśli zrobisz błąd; po prostu oceniamy odpowiedź.
‘‘‘
Pytanie: {question}
Wzorcowa odpowiedź: {target}
Przewidywana odpowiedź: {predicted_answer}
‘‘‘
Oceń przewidywaną odpowiedź na podstawie:
A: CORRECT (POPRAWNA)
B: INCORRECT (NIEPOPRAWNA)
C: NOT_ATTEMPTED (NIEPODJĘTA)
Zwróć tylko literę "A", "B" lub "C", bez żadnego dodatkowego tekstu.
""".strip()


<h3>Evaluating Whether the Answers are CORRECT, INCORRECT, or NOT ATTEMPTED</h3>

In [ ]:
QUESTION_DATASET_PATH = "../data/output/bg/triviaqaSimpleQA-BGMistral.csv"
DESTINATION_DATASET_PATH = "../data/output/bg/triviaqaSimpleQA-BGMistral-graded.csv"
GRADER_TEMPLATE = GRADER_TEMPLATE_PL

In [ ]:
# Initialise the API key (TODO: change to use dotenv for security)
api_key = "" 
base_url = "https://api.aimlapi.com"

# Create an OpenAI client instance
client = OpenAI(api_key=api_key)

def get_llm_answer(question, target, predicted_answer):
        messages = [ {"role": "user", "content": GRADER_TEMPLATE.format(question=question, target=target, predicted_answer=predicted_answer)}]

        response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=100,
                temperature=0.0, # 0.0 is deterministic
            )

        return response.choices[0].message.content.strip()

In [15]:
bg_trivia = pd.read_csv(QUESTION_DATASET_PATH)

questions = bg_trivia["Question"]
gold_targets = bg_trivia["Answers"]
predicted_answers = bg_trivia["Predicted answers"] 
results = []

for i in range(len(questions)):
    classification = get_llm_answer(questions[i], gold_targets[i], predicted_answers[i])
    results.append({"Question number": i + 1, "Classification": classification})

results_df = pd.DataFrame(results)
results_df.to_csv(DESTINATION_DATASET_PATH, index=False)

Index(['Question', 'Answers', 'Predicted answers'], dtype='object')
